In [ ]:
!ls
!pwd

download_audio.py	  fluencybank_labels.csv  requirements.txt	train
ex1.py			  FluencyBank.png	  SEP-28k_episodes.csv	val
extract_clips.py	  fluency_bank.zip	  SEP-28k_labels.csv
fluency_bank		  preprocessing.py	  TEAM10
fluencybank_episodes.csv  __pycache__		  test
/work


# fluencyBank

In [ ]:
!ls fluency_bank/

FluencyBank_024_111.wav  FluencyBank_096_84.wav   FluencyBank_173_56.wav
FluencyBank_024_112.wav  FluencyBank_096_85.wav   FluencyBank_173_57.wav
FluencyBank_024_113.wav  FluencyBank_096_86.wav   FluencyBank_173_58.wav
FluencyBank_024_114.wav  FluencyBank_096_87.wav   FluencyBank_173_59.wav
FluencyBank_024_115.wav  FluencyBank_096_88.wav   FluencyBank_173_5.wav
FluencyBank_024_116.wav  FluencyBank_096_89.wav   FluencyBank_173_60.wav
FluencyBank_024_117.wav  FluencyBank_096_8.wav	  FluencyBank_173_61.wav
FluencyBank_024_118.wav  FluencyBank_096_90.wav   FluencyBank_173_62.wav
FluencyBank_024_119.wav  FluencyBank_096_91.wav   FluencyBank_173_63.wav
FluencyBank_024_11.wav	 FluencyBank_096_92.wav   FluencyBank_173_64.wav
FluencyBank_024_120.wav  FluencyBank_096_93.wav   FluencyBank_173_65.wav
FluencyBank_024_121.wav  FluencyBank_096_94.wav   FluencyBank_173_66.wav
FluencyBank_024_122.wav  FluencyBank_096_9.wav	  FluencyBank_173_67.wav
FluencyBank_024_123.wav  FluencyBank_097_0.wav	  Fluenc

In [ ]:
import os

# Specify the directory where your .wav files are located
directory = './fluency_bank_clips_all'

# Initialize a counter variable
count = 0

# Iterate over the files in the directory
for filename in os.listdir(directory):
    if filename.endswith('.wav'):
        count += 1

print(f"Number of .wav files in the folder: {count}")

Number of .wav files in the folder: 4144


In [1]:
import pandas as pd

df = pd.read_csv('fluencybank_labels.csv')
df.head()

,Show,EpId,ClipId,Start,Stop,Unsure,PoorAudioQuality,Prolongation,Block,SoundRep,WordRep,DifficultToUnderstand,Interjection,NoStutteredWords,NaturalPause,Music,NoSpeech
0,FluencyBank,10,0,88960,136960,0,0,0,1,0,0,0,0,2,0,0,0
1,FluencyBank,10,1,1271520,1319520,0,0,0,0,0,0,0,0,3,0,0,0
2,FluencyBank,10,2,1813760,1861760,0,0,1,0,0,0,0,0,2,0,0,0
3,FluencyBank,10,3,1842720,1890720,0,0,1,0,0,0,0,0,2,1,0,0
4,FluencyBank,10,4,1893280,1941280,0,0,0,0,0,3,0,0,0,0,0,0


In [ ]:
import os
import tensorflow as tf

# Specify the source directory where your files are located
folder_path = './fluency_bank_Prolongation/'

for filename in os.listdir(folder_path):

    path_parts = tf.strings.split(filename, '/')
    path_end = path_parts[-1]
    file_parts = tf.strings.split(path_end, '_')
    pod_id = int(file_parts[-2])
    clip_id = int(tf.strings.split(file_parts[-1], '.')[-2])
    features = df[(df['EpId'] == pod_id) & (df['ClipId'] == clip_id)].to_numpy()[0][5:]
    break
features

array([0, 0, 0, 1, 0, 0, 1, 0, 2, 1, 0, 0], dtype=object)

In [ ]:
Prolongation_num = df[df['Prolongation'] > 1]
Block_num = df[df['Block'] != 0]
SoundRep_num = df[df['SoundRep'] != 0]
NaturalPause_num = df[df['Interjection'] != 0]
tmp = df[(df['NoStutteredWords'] > 2)]

len(Prolongation_num), len(Block_num), len(SoundRep_num), len(NaturalPause_num), len(df), len(tmp)

(338, 1486, 1148, 1658, 4144, 1438)

In [ ]:
import os
import random
import shutil

# Specify the source directory where your files are located
source_directory = './fluency_bank_clips_all/'

# Specify the destination directories for the split files
destination_directory = './fluency_bank_Prolongation'

# Create the destination directories if they don't exist
if not os.path.exists(destination_directory):
    os.makedirs(destination_directory)

files = os.listdir(source_directory)

for filename in files:
    source_path = os.path.join(source_directory, filename)
    destination_path = os.path.join(destination_directory, filename)
    shutil.copy2(source_path, destination_path)

print("Files split and copied successfully!")

Files split and copied successfully!


In [ ]:
import os

# Specify the directory where your .wav files are located
directory = './fluency_bank_Prolongation'

# Initialize a counter variable
count = 0

# Iterate over the files in the directory
for filename in os.listdir(directory):
    if filename.endswith('.wav'):
        count += 1

print(f"Number of .wav files in the folder: {count}")

Number of .wav files in the folder: 4144


In [ ]:
import os
import tensorflow as tf

# Specify the source directory where your files are located
folder_path = './fluency_bank_Prolongation/'

for filename in os.listdir(folder_path):

    path_parts = tf.strings.split(filename, '/')
    path_end = path_parts[-1]
    file_parts = tf.strings.split(path_end, '_')
    pod_id = int(file_parts[-2])
    clip_id = int(tf.strings.split(file_parts[-1], '.')[-2])
    features = df[(df['EpId'] == pod_id) & (df['ClipId'] == clip_id)].to_numpy()[0][5:]
    
    if features[2] != 0:
        old_file_path = os.path.join(folder_path, filename)
        new_filename = "prolongation_" + filename  # Replace "new_name_" with your desired naming convention
        new_file_path = os.path.join(folder_path, new_filename)
        os.rename(old_file_path, new_file_path)

    else:
        old_file_path = os.path.join(folder_path, filename)
        new_filename = "noprolongation_" + filename  # Replace "new_name_" with your desired naming convention
        new_file_path = os.path.join(folder_path, new_filename)
        os.rename(old_file_path, new_file_path)
       

In [ ]:
import os

# Specify the directory where your .wav files are located
directory = './fluency_bank_Prolongation'

# Initialize a counter variable
count = 0
count1 = 0
# Iterate over the files in the directory
for filename in os.listdir(directory):
    if filename.startswith('prolongation'):
        count += 1
    count1 += 1

print(f"Number of prolongation files in the folder: {count} and tot {count1}")

Number of prolongation files in the folder: 936 and tot 4144


In [ ]:
import os
import random
import shutil

# Specify the source directory where your files are located
source_directory = './fluency_bank_Prolongation'

# Specify the destination directories for the split files
destination_directory_1 = './train'
destination_directory_2 = './val'
destination_directory_3 = './test'

# Create the destination directories if they don't exist
for directory in [destination_directory_1, destination_directory_2, destination_directory_3]:
    if not os.path.exists(directory):
        os.makedirs(directory)

# Shuffle the files in the source directory
files = os.listdir(source_directory)
random.shuffle(files)

# Define the sizes for each split
split_sizes = [0.8, 0.1, 0.1]  # 60% for directory_1, 30% for directory_2, 10% for directory_3

# Calculate the number of files for each split
total_files = len(files)
split_counts = [int(size * total_files) for size in split_sizes]

# Split the files into different lists based on the split counts
file_splits = [files[:split_counts[0]], files[split_counts[0]:split_counts[0] + split_counts[1]], files[-split_counts[2]:]]

# Move the files to the destination directories
for i, file_split in enumerate(file_splits):
    if i == 0:
        destination_directory = destination_directory_1
    elif i == 1:
        destination_directory = destination_directory_2
    else:
        destination_directory = destination_directory_3

    for filename in file_split:
        source_path = os.path.join(source_directory, filename)
        destination_path = os.path.join(destination_directory, filename)
        shutil.copy2(source_path, destination_path)

print("Files split and copied successfully!")


Files split and copied successfully!


In [ ]:
import os
import tensorflow as tf

# Specify the directory where your .wav files are located
directory1 = './train_fluencybank'
directory2 = './val_fluencybank'
directory3 = './test_fluencybank'

# Initialize a counter variable
count_train = 0
count_val = 0
count_test = 0

count_train_tot = 0
count_val_tot = 0
count_test_tot = 0

# import os

# # Specify the folder path where the WAV files are located
# folder_path = "/path/to/folder"

# # List of WAV files to be deleted (provide the file names without the folder path)
# files_to_delete = ["file1.wav", "file2.wav", "file3.wav"]

# # Iterate over the files in the folder
# for filename in os.listdir(folder_path):
#     file_path = os.path.join(folder_path, filename)
    
#     # Check if the file is a WAV file and is in the list of files to delete
#     if filename.endswith(".wav") and filename in files_to_delete:
#         # Delete the file
#         os.remove(file_path)
#         print(f"Deleted {filename}")

# print("Deletion complete.")

# Iterate over the files in the directory
for filename in os.listdir(directory1):
    audio_binary = tf.io.read_file(f'{directory1}/{filename}')
    audio, sampling_rate = tf.audio.decode_wav(audio_binary) 
    
    if ((audio.shape != (48000, 1)) or (sampling_rate.numpy() != 16000)):
        print(audio.shape, sampling_rate.numpy(), filename)
        print('error 1')
        file_path = os.path.join(directory1, filename)
        os.remove(file_path)

    if filename.startswith('prolongation'):
        count_train += 1

    if filename.endswith('.wav'):
        count_train_tot += 1

# Iterate over the files in the directory
for filename in os.listdir(directory2):
    audio_binary = tf.io.read_file(f'{directory2}/{filename}')
    audio, sampling_rate = tf.audio.decode_wav(audio_binary) 

    if  ((audio.shape != (48000, 1)) | (sampling_rate.numpy() != 16000)):
        print(audio.shape, sampling_rate.numpy(), filename)
        print('error 2')
        file_path = os.path.join(directory2, filename)
        os.remove(file_path)

    if filename.startswith('prolongation'):
        count_val += 1

    if filename.endswith('.wav'):
        count_val_tot += 1

# Iterate over the files in the directory
for filename in os.listdir(directory3):
    audio_binary = tf.io.read_file(f'{directory3}/{filename}')
    audio, sampling_rate = tf.audio.decode_wav(audio_binary)

    if  ((audio.shape != (48000, 1)) | (sampling_rate.numpy() != 16000)):
        print(audio.shape, sampling_rate.numpy(), filename)
        print('error 3')
        file_path = os.path.join(directory3, filename)
        os.remove(file_path)

    if filename.startswith('prolongation'):
        count_test += 1
        
    if filename.endswith('.wav'):
        count_test_tot += 1

print(f"Number of .wav files in the train folder: {count_train} and tot is {count_train_tot}")
print(f"Number of .wav files in the val folder: {count_val} and tot is {count_val_tot}")
print(f"Number of .wav files in the test folder: {count_test} and tot is {count_test_tot}")

Number of .wav files in the train folder: 744 and tot is 3288
Number of .wav files in the val folder: 91 and tot is 411
Number of .wav files in the test folder: 97 and tot is 411


In [ ]:
import os
import tensorflow as tf

# Specify the directory where your .wav files are located
directory1 = './train_fluencybank'
directory2 = './val_fluencybank'
directory3 = './test_fluencybank'

# Initialize a counter variable
count_train = 0
count_val = 0
count_test = 0

count_train_tot = 0
count_val_tot = 0
count_test_tot = 0

# import os

# # Specify the folder path where the WAV files are located
# folder_path = "/path/to/folder"

# # List of WAV files to be deleted (provide the file names without the folder path)
# files_to_delete = ["file1.wav", "file2.wav", "file3.wav"]

# # Iterate over the files in the folder
# for filename in os.listdir(folder_path):
#     file_path = os.path.join(folder_path, filename)
    
#     # Check if the file is a WAV file and is in the list of files to delete
#     if filename.endswith(".wav") and filename in files_to_delete:
#         # Delete the file
#         os.remove(file_path)
#         print(f"Deleted {filename}")

# print("Deletion complete.")

# Iterate over the files in the directory
for filename in os.listdir(directory1):
    audio_binary = tf.io.read_file(f'{directory1}/{filename}')
    audio, sampling_rate = tf.audio.decode_wav(audio_binary) 
    
    if ((audio.shape != (48000, 1)) or (sampling_rate.numpy() != 16000)):
        print(audio.shape, sampling_rate.numpy(), filename)
        print('error 1')
        file_path = os.path.join(directory1, filename)
        os.remove(file_path)

    if filename.startswith('prolongation'):
        count_train += 1

    if filename.endswith('.wav'):
        count_train_tot += 1

# Iterate over the files in the directory
for filename in os.listdir(directory2):
    audio_binary = tf.io.read_file(f'{directory2}/{filename}')
    audio, sampling_rate = tf.audio.decode_wav(audio_binary) 

    if  ((audio.shape != (48000, 1)) | (sampling_rate.numpy() != 16000)):
        print(audio.shape, sampling_rate.numpy(), filename)
        print('error 2')
        file_path = os.path.join(directory2, filename)
        os.remove(file_path)

    if filename.startswith('prolongation'):
        count_val += 1

    if filename.endswith('.wav'):
        count_val_tot += 1

# Iterate over the files in the directory
for filename in os.listdir(directory3):
    audio_binary = tf.io.read_file(f'{directory3}/{filename}')
    audio, sampling_rate = tf.audio.decode_wav(audio_binary)

    if  ((audio.shape != (48000, 1)) | (sampling_rate.numpy() != 16000)):
        print(audio.shape, sampling_rate.numpy(), filename)
        print('error 3')
        file_path = os.path.join(directory3, filename)
        os.remove(file_path)

    if filename.startswith('prolongation'):
        count_test += 1
        
    if filename.endswith('.wav'):
        count_test_tot += 1

print(f"Number of .wav files in the train folder: {count_train} and tot is {count_train_tot}")
print(f"Number of .wav files in the val folder: {count_val} and tot is {count_val_tot}")
print(f"Number of .wav files in the test folder: {count_test} and tot is {count_test_tot}")

Number of .wav files in the train folder: 744 and tot is 3288
Number of .wav files in the val folder: 91 and tot is 411
Number of .wav files in the test folder: 97 and tot is 411


# sep28

In [ ]:
import os
import shutil

def collect_files(source_folders, destination_folder):
    for folder in source_folders:
        for root, dirs, files in os.walk(folder):
            for file in files:
                source_path = os.path.join(root, file)
                destination_path = os.path.join(destination_folder, file)
                shutil.move(source_path, destination_folder)
                print(f"Moved {source_path} to {destination_path}")

# Example usage
source_folders = ['./sep28_clips_by_folder/']
destination_folder = './temp'

collect_files(source_folders, destination_folder)


Moved ./sep28_clips_by_folder/MyStutteringLife/35/MyStutteringLife_35_164.wav to ./temp/MyStutteringLife_35_164.wav
Moved ./sep28_clips_by_folder/MyStutteringLife/35/MyStutteringLife_35_118.wav to ./temp/MyStutteringLife_35_118.wav
Moved ./sep28_clips_by_folder/MyStutteringLife/35/MyStutteringLife_35_6.wav to ./temp/MyStutteringLife_35_6.wav
Moved ./sep28_clips_by_folder/MyStutteringLife/35/MyStutteringLife_35_201.wav to ./temp/MyStutteringLife_35_201.wav
Moved ./sep28_clips_by_folder/MyStutteringLife/35/MyStutteringLife_35_63.wav to ./temp/MyStutteringLife_35_63.wav
Moved ./sep28_clips_by_folder/MyStutteringLife/35/MyStutteringLife_35_116.wav to ./temp/MyStutteringLife_35_116.wav
Moved ./sep28_clips_by_folder/MyStutteringLife/35/MyStutteringLife_35_8.wav to ./temp/MyStutteringLife_35_8.wav
Moved ./sep28_clips_by_folder/MyStutteringLife/35/MyStutteringLife_35_206.wav to ./temp/MyStutteringLife_35_206.wav
Moved ./sep28_clips_by_folder/MyStutteringLife/35/MyStutteringLife_35_64.wav to ./

In [ ]:
!ls ./train_sep28/noprolongation*

./train_sep28/noprolongation_WomenWhoStutter_83_30.wav
./train_sep28/noprolongation_WomenWhoStutter_83_32.wav
./train_sep28/noprolongation_WomenWhoStutter_83_33.wav
./train_sep28/noprolongation_WomenWhoStutter_83_34.wav
./train_sep28/noprolongation_WomenWhoStutter_83_35.wav
./train_sep28/noprolongation_WomenWhoStutter_83_37.wav
./train_sep28/noprolongation_WomenWhoStutter_83_38.wav
./train_sep28/noprolongation_WomenWhoStutter_83_3.wav
./train_sep28/noprolongation_WomenWhoStutter_83_4.wav
./train_sep28/noprolongation_WomenWhoStutter_8_35.wav
./train_sep28/noprolongation_WomenWhoStutter_8_37.wav
./train_sep28/noprolongation_WomenWhoStutter_83_7.wav
./train_sep28/noprolongation_WomenWhoStutter_8_38.wav
./train_sep28/noprolongation_WomenWhoStutter_83_8.wav
./train_sep28/noprolongation_WomenWhoStutter_83_9.wav
./train_sep28/noprolongation_WomenWhoStutter_8_3.wav
./train_sep28/noprolongation_WomenWhoStutter_84_0.wav
./train_sep28/noprolongation_WomenWhoStutter_84_10.wav
./train_sep28/noprolo

In [ ]:
import os

# Specify the directory where your .wav files are located
directory = './sep28_clips_all'

# Initialize a counter variable
count = 0

# Iterate over the files in the directory
for filename in os.listdir(directory):
    if filename.endswith('.wav'):
        count += 1

print(f"Number of .wav files in the folder: {count}")

Number of .wav files in the folder: 21856


In [ ]:
import os
import random
import shutil

# Specify the source directory where your files are located
source_directory = './sep28_clips_all/'

# Specify the destination directories for the split files
destination_directory = './sep28_Prolongation'

# Create the destination directories if they don't exist
if not os.path.exists(destination_directory):
    os.makedirs(destination_directory)

files = os.listdir(source_directory)

for filename in files:
    source_path = os.path.join(source_directory, filename)
    destination_path = os.path.join(destination_directory, filename)
    shutil.move(source_path, destination_path)

print("Files split and copied successfully!")

Files split and copied successfully!


In [ ]:
import os

# Specify the directory where your .wav files are located
directory = './sep28_Prolongation'

# Initialize a counter variable
count = 0

# Iterate over the files in the directory
for filename in os.listdir(directory):
    if filename.endswith('.wav'):
        count += 1

print(f"Number of .wav files in the folder: {count}")

Number of .wav files in the folder: 21856


In [ ]:
import pandas as pd

df_sep = pd.read_csv('SEP_28k_labels.csv')
df_sep.head()

,Show,EpId,ClipId,Start,Stop,Unsure,PoorAudioQuality,Prolongation,Block,SoundRep,WordRep,DifficultToUnderstand,Interjection,NoStutteredWords,NaturalPause,Music,NoSpeech
0,HeStutters,0,0,31900320,31948320,0,0,0,0,0,0,0,0,3,1,0,0
1,HeStutters,0,1,31977120,32025120,0,0,0,0,0,0,0,0,3,1,0,0
2,HeStutters,0,2,34809760,34857760,0,0,0,0,0,0,0,0,3,0,0,0
3,HeStutters,0,3,35200640,35248640,0,0,1,0,0,0,0,0,2,0,0,0
4,HeStutters,0,4,35721920,35769920,0,0,0,0,0,0,0,0,3,0,0,0


In [ ]:
Prolongation_num = df_sep[df_sep['Prolongation'] != 0]
Block_num = df_sep[df_sep['Block'] != 0]
SoundRep_num = df_sep[df_sep['SoundRep'] != 0]
NaturalPause_num = df_sep[df_sep['Interjection'] != 0]
tmp = df_sep[(df_sep['NoStutteredWords'] > 2)]

len(Prolongation_num), len(Block_num), len(SoundRep_num), len(NaturalPause_num), len(df_sep), len(tmp)

(8546, 11970, 5614, 9658, 28177, 8586)

In [ ]:
import os
import tensorflow as tf

# Specify the source directory where your files are located
folder_path = './sep28_Prolongation/'

for filename in os.listdir(folder_path):

    path_parts = tf.strings.split(filename, '/')
    path_end = path_parts[-1]
    file_parts = tf.strings.split(path_end, '_')
    pod_id = int(file_parts[-2])
    clip_id = int(tf.strings.split(file_parts[-1], '.')[-2])
    features = df_sep[(df_sep['EpId'] == pod_id) & (df_sep['ClipId'] == clip_id)].to_numpy()[0][5:]
    
    if features[2] != 0:
        old_file_path = os.path.join(folder_path, filename)
        new_filename = "prolongation_" + filename  # Replace "new_name_" with your desired naming convention
        new_file_path = os.path.join(folder_path, new_filename)
        os.rename(old_file_path, new_file_path)

    else:
        old_file_path = os.path.join(folder_path, filename)
        new_filename = "noprolongation_" + filename  # Replace "new_name_" with your desired naming convention
        new_file_path = os.path.join(folder_path, new_filename)
        os.rename(old_file_path, new_file_path)

2023-05-24 20:08:57.862475: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-05-24 20:08:57.862530: W tensorflow/stream_executor/cuda/cuda_driver.cc:263] failed call to cuInit: UNKNOWN ERROR (303)
2023-05-24 20:08:57.862556: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (p-5c74a7a1-d505-4b2b-a4fa-efea0c0f20da): /proc/driver/nvidia/version does not exist
2023-05-24 20:08:57.862880: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [ ]:
import os

# Specify the directory where your .wav files are located
directory = './sep28_Prolongation/'

# Initialize a counter variable
count = 0
count1 = 0
# Iterate over the files in the directory
for filename in os.listdir(directory):
    if filename.startswith('prolongation'):
        count += 1
    count1 += 1

print(f"Number of prolongation files in the folder: {count} and tot {count1}")

Number of prolongation files in the folder: 6845 and tot 21836


In [ ]:
directory1 = './sep28_Prolongation'

# Iterate over the files in the directory
for filename in os.listdir(directory1):
    audio_binary = tf.io.read_file(f'{directory1}/{filename}')
    audio, sampling_rate = tf.audio.decode_wav(audio_binary) 
    
    if ((audio.shape != (48000, 1)) or (sampling_rate.numpy() != 16000)):
        print(audio.shape, sampling_rate.numpy(), filename)
        print('error 1')
        file_path = os.path.join(directory1, filename)
        os.remove(file_path)

(42131, 1) 16000 noprolongation_WomenWhoStutter_47_1.wav
error 1
(43078, 1) 16000 noprolongation_MyStutteringLife_18_1.wav
error 1
(40800, 1) 16000 noprolongation_WomenWhoStutter_73_0.wav
error 1
(41280, 1) 16000 noprolongation_WomenWhoStutter_101_0.wav
error 1
(41440, 1) 16000 noprolongation_WomenWhoStutter_97_0.wav
error 1
(41360, 1) 16000 noprolongation_WomenWhoStutter_80_9.wav
error 1
(46400, 1) 16000 noprolongation_HeStutters_17_0.wav
error 1
(41248, 1) 16000 noprolongation_WomenWhoStutter_101_35.wav
error 1
(43085, 1) 16000 noprolongation_MyStutteringLife_23_3.wav
error 1
(40960, 1) 16000 noprolongation_WomenWhoStutter_87_0.wav
error 1
(46080, 1) 16000 noprolongation_IStutterSoWhat_2_37.wav
error 1
(40800, 1) 16000 noprolongation_WomenWhoStutter_86_0.wav
error 1
(40979, 1) 16000 noprolongation_MyStutteringLife_0_10.wav
error 1
(45296, 1) 16000 prolongation_StutterTalk_5_8.wav
error 1
(40800, 1) 16000 noprolongation_WomenWhoStutter_85_0.wav
error 1
(40640, 1) 16000 noprolongation_

In [ ]:
import random
import shutil

# Specify the source directory where your files are located
source_directory = './sep28_Prolongation'

# Specify the destination directories for the split files
destination_directory_1 = './train_sep28'
destination_directory_2 = './val_sep28'
destination_directory_3 = './test_sep28'

# Create the destination directories if they don't exist
for directory in [destination_directory_1, destination_directory_2, destination_directory_3]:
    if not os.path.exists(directory):
        os.makedirs(directory)

# Shuffle the files in the source directory
files = os.listdir(source_directory)
random.shuffle(files)

# Define the sizes for each split
split_sizes = [0.8, 0.1, 0.1]  # 60% for directory_1, 30% for directory_2, 10% for directory_3

# Calculate the number of files for each split
total_files = len(files)
split_counts = [int(size * total_files) for size in split_sizes]

# Split the files into different lists based on the split counts
file_splits = [files[:split_counts[0]], files[split_counts[0]:split_counts[0] + split_counts[1]], files[-split_counts[2]:]]

# Move the files to the destination directories
for i, file_split in enumerate(file_splits):
    if i == 0:
        destination_directory = destination_directory_1
    elif i == 1:
        destination_directory = destination_directory_2
    else:
        destination_directory = destination_directory_3

    for filename in file_split:
        source_path = os.path.join(source_directory, filename)
        destination_path = os.path.join(destination_directory, filename)
        shutil.copy2(source_path, destination_path)

print("Files split and copied successfully!")

Files split and copied successfully!


In [ ]:
import os
import tensorflow as tf

# Specify the directory where your .wav files are located
directory1 = './train_sep28'
directory2 = './val_sep28'
directory3 = './test_sep28'

# Initialize a counter variable
count_train = 0
count_val = 0
count_test = 0

count_train_tot = 0
count_val_tot = 0
count_test_tot = 0

# Iterate over the files in the directory
for filename in os.listdir(directory1):
    audio_binary = tf.io.read_file(f'{directory1}/{filename}')
    audio, sampling_rate = tf.audio.decode_wav(audio_binary) 
    
    if ((audio.shape != (48000, 1)) or (sampling_rate.numpy() != 16000)):
        print(audio.shape, sampling_rate.numpy(), filename)
        print('error 1')
        file_path = os.path.join(directory1, filename)
        os.remove(file_path)

    if filename.startswith('prolongation'):
        count_train += 1

    if filename.endswith('.wav'):
        count_train_tot += 1

# Iterate over the files in the directory
for filename in os.listdir(directory2):
    audio_binary = tf.io.read_file(f'{directory2}/{filename}')
    audio, sampling_rate = tf.audio.decode_wav(audio_binary) 

    if  ((audio.shape != (48000, 1)) | (sampling_rate.numpy() != 16000)):
        print(audio.shape, sampling_rate.numpy(), filename)
        print('error 2')
        file_path = os.path.join(directory2, filename)
        os.remove(file_path)

    if filename.startswith('prolongation'):
        count_val += 1

    if filename.endswith('.wav'):
        count_val_tot += 1

# Iterate over the files in the directory
for filename in os.listdir(directory3):
    audio_binary = tf.io.read_file(f'{directory3}/{filename}')
    audio, sampling_rate = tf.audio.decode_wav(audio_binary)

    if  ((audio.shape != (48000, 1)) | (sampling_rate.numpy() != 16000)):
        print(audio.shape, sampling_rate.numpy(), filename)
        print('error 3')
        file_path = os.path.join(directory3, filename)
        os.remove(file_path)

    if filename.startswith('prolongation'):
        count_test += 1
        
    if filename.endswith('.wav'):
        count_test_tot += 1

print(f"Number of .wav files in the train folder: {count_train} and tot is {count_train_tot}")
print(f"Number of .wav files in the val folder: {count_val} and tot is {count_val_tot}")
print(f"Number of .wav files in the test folder: {count_test} and tot is {count_test_tot}")

Number of .wav files in the train folder: 5454 and tot is 17468
Number of .wav files in the val folder: 687 and tot is 2183
Number of .wav files in the test folder: 704 and tot is 2183


In [ ]:
# Specify the directory where your .wav files are located
directory1 = './train_sep28'
directory2 = './val_sep28'
directory3 = './test_sep28'

# Initialize a counter variable
count_train = 0
count_val = 0
count_test = 0

count_train_tot = 0
count_val_tot = 0
count_test_tot = 0

# Iterate over the files in the directory
for filename in os.listdir(directory1):
    if filename.startswith('prolongation'):
        count_train += 1

    if filename.endswith('.wav'):
        count_train_tot += 1
    if not filename.endswith('.wav'):
        print('error')
# Iterate over the files in the directory
for filename in os.listdir(directory2):
    if filename.startswith('prolongation'):
        count_val += 1

    if filename.endswith('.wav'):
        count_val_tot += 1
    if not filename.endswith('.wav'):
        print('error')
# Iterate over the files in the directory
for filename in os.listdir(directory3):
    if filename.startswith('prolongation'):
        count_test += 1
        
    if filename.endswith('.wav'):
        count_test_tot += 1

    if not filename.startswith('prolongation') and not filename.startswith('noprolongation'):
        print(filename)
print('**********Sep28 dataset**********')
print(f"Number of prolongation files in the train folder: {count_train} and tot is {count_train_tot}")
print(f"Number of prolongation files in the val folder: {count_val} and tot is {count_val_tot}")
print(f"Number of prolongation files in the test folder: {count_test} and tot is {count_test_tot}")

**********Sep28 dataset**********
Number of prolongation files in the train folder: 5454 and tot is 17468
Number of prolongation files in the val folder: 687 and tot is 2183
Number of prolongation files in the test folder: 704 and tot is 2183


In [ ]:
!ls ./train_sep28/

prolongation_WomenWhoStutter_50_36.wav
prolongation_WomenWhoStutter_50_370.wav
prolongation_WomenWhoStutter_50_373.wav
prolongation_WomenWhoStutter_50_375.wav
prolongation_WomenWhoStutter_50_376.wav
prolongation_WomenWhoStutter_50_377.wav
prolongation_WomenWhoStutter_50_382.wav
prolongation_WomenWhoStutter_50_387.wav
prolongation_WomenWhoStutter_50_38.wav
prolongation_WomenWhoStutter_50_390.wav
prolongation_WomenWhoStutter_50_393.wav
prolongation_WomenWhoStutter_50_395.wav
prolongation_WomenWhoStutter_50_396.wav
prolongation_WomenWhoStutter_50_397.wav
prolongation_WomenWhoStutter_50_401.wav
prolongation_WomenWhoStutter_50_404.wav
prolongation_WomenWhoStutter_50_405.wav
prolongation_WomenWhoStutter_50_407.wav
prolongation_WomenWhoStutter_50_409.wav
prolongation_WomenWhoStutter_50_410.wav
prolongation_WomenWhoStutter_50_411.wav
prolongation_WomenWhoStutter_50_412.wav
prolongation_WomenWhoStutter_50_416.wav
prolongation_WomenWhoStutter_50_418.wav
prolongation_WomenWhoStutter_50_41.wav
pro

# Stutter dataset

SEP28

In [3]:
!unzip sep28_all_together

  inflating: sep28_all_together/WomenWhoStutter_89_35.wav  
  inflating: sep28_all_together/WomenWhoStutter_89_36.wav  
  inflating: sep28_all_together/WomenWhoStutter_89_37.wav  
  inflating: sep28_all_together/WomenWhoStutter_89_38.wav  
  inflating: sep28_all_together/WomenWhoStutter_89_39.wav  
  inflating: sep28_all_together/WomenWhoStutter_89_4.wav  
  inflating: sep28_all_together/WomenWhoStutter_89_5.wav  
  inflating: sep28_all_together/WomenWhoStutter_89_6.wav  
  inflating: sep28_all_together/WomenWhoStutter_89_7.wav  
  inflating: sep28_all_together/WomenWhoStutter_89_8.wav  
  inflating: sep28_all_together/WomenWhoStutter_89_9.wav  
  inflating: sep28_all_together/WomenWhoStutter_8_0.wav  
  inflating: sep28_all_together/WomenWhoStutter_8_1.wav  
  inflating: sep28_all_together/WomenWhoStutter_8_10.wav  
  inflating: sep28_all_together/WomenWhoStutter_8_11.wav  
  inflating: sep28_all_together/WomenWhoStutter_8_12.wav  
  inflating: sep28_all_together/WomenWhoStutter_8_13.

In [1]:
!ls ./sep28_all_together/

stutter_IStutterSoWhat_1_66.wav        stutter_WomenWhoStutter_87_118.wav
stutter_IStutterSoWhat_1_67.wav        stutter_WomenWhoStutter_87_119.wav
stutter_IStutterSoWhat_1_68.wav        stutter_WomenWhoStutter_87_11.wav
stutter_IStutterSoWhat_1_69.wav        stutter_WomenWhoStutter_87_120.wav
stutter_IStutterSoWhat_1_6.wav	       stutter_WomenWhoStutter_87_121.wav
stutter_IStutterSoWhat_1_70.wav        stutter_WomenWhoStutter_87_122.wav
stutter_IStutterSoWhat_1_71.wav        stutter_WomenWhoStutter_87_123.wav
stutter_IStutterSoWhat_1_73.wav        stutter_WomenWhoStutter_87_124.wav
stutter_IStutterSoWhat_1_74.wav        stutter_WomenWhoStutter_87_126.wav
stutter_IStutterSoWhat_1_75.wav        stutter_WomenWhoStutter_87_127.wav
stutter_IStutterSoWhat_1_76.wav        stutter_WomenWhoStutter_87_128.wav
stutter_IStutterSoWhat_1_77.wav        stutter_WomenWhoStutter_87_129.wav
stutter_IStutterSoWhat_1_78.wav        stutter_WomenWhoStutter_87_12.wav
stutter_IStutterSoWhat_1_79.wav        st

In [4]:
import pandas as pd

df_sep = pd.read_csv('SEP_28k_labels.csv')
df_sep.head()

,Show,EpId,ClipId,Start,Stop,Unsure,PoorAudioQuality,Prolongation,Block,SoundRep,WordRep,DifficultToUnderstand,Interjection,NoStutteredWords,NaturalPause,Music,NoSpeech
0,HeStutters,0,0,31900320,31948320,0,0,0,0,0,0,0,0,3,1,0,0
1,HeStutters,0,1,31977120,32025120,0,0,0,0,0,0,0,0,3,1,0,0
2,HeStutters,0,2,34809760,34857760,0,0,0,0,0,0,0,0,3,0,0,0
3,HeStutters,0,3,35200640,35248640,0,0,1,0,0,0,0,0,2,0,0,0
4,HeStutters,0,4,35721920,35769920,0,0,0,0,0,0,0,0,3,0,0,0


In [3]:
df_sep[(df_sep['EpId'] == 19) & (df_sep['ClipId'] == 30)]

,Show,EpId,ClipId,Start,Stop,Unsure,PoorAudioQuality,Prolongation,Block,SoundRep,WordRep,DifficultToUnderstand,Interjection,NoStutteredWords,NaturalPause,Music,NoSpeech
2627,HeStutters,19,30,1249280,1297280,0,0,0,0,0,0,0,2,3,1,0,0
6234,MyStutteringLife,19,30,27917920,27965920,0,0,0,0,0,0,1,0,3,2,0,0
8774,StrongVoices,19,30,19366560,19414560,0,1,0,1,0,0,1,0,2,0,0,0
11067,StutterTalk,19,30,32482080,32530080,0,0,0,0,0,0,0,0,3,0,0,0
15791,StutteringIsCool,19,30,21815680,21863680,0,0,0,1,0,0,0,1,2,1,0,0
20876,WomenWhoStutter,19,30,34112480,34160480,0,0,0,0,0,3,0,0,0,0,0,0


In [13]:
Prolongation_num = df_sep[df_sep['Prolongation'] != 0]
Block_num = df_sep[df_sep['Block'] != 0]
SoundRep_num = df_sep[df_sep['SoundRep'] != 0]
NaturalPause_num = df_sep[df_sep['Interjection'] != 0]
tmp = df_sep[(df_sep['NoStutteredWords'] == 3) & (df_sep['Prolongation'] != 0)& (df_sep['Interjection'] != 0)]

len(Prolongation_num), len(Block_num), len(SoundRep_num), len(NaturalPause_num), len(df_sep), len(tmp)

(8546, 11970, 5614, 9658, 28177, 87)

In [16]:
print(df_sep[(df_sep['Show'] == 'WomenWhoStutter') & (df_sep['EpId'] == 99) & (df_sep['ClipId'] == 99)]['NoStutteredWords'])

27324    2
Name: NoStutteredWords, dtype: int64


In [18]:
import os
import tensorflow as tf

# Specify the source directory where your files are located
folder_path = './sep28_all_together/'
count = 0
for filename in os.listdir(folder_path):

    path_parts = tf.strings.split(filename, '/')
    path_end = path_parts[-1]
    file_parts = tf.strings.split(path_end, '_')
    show_name = file_parts[0].numpy().decode('utf-8')
    pod_id = int(file_parts[-2])
    clip_id = int(tf.strings.split(file_parts[-1], '.')[-2])
    features = df_sep[(df_sep['Show'] == show_name) & (df_sep['EpId'] == pod_id) & (df_sep['ClipId'] == clip_id)].to_numpy()[0][5:]
    
    print(pod_id,clip_id,show_name)
    print(df_sep[(df_sep['Show'] == show_name) &(df_sep['EpId'] == pod_id) & (df_sep['ClipId'] == clip_id)]['NoStutteredWords'])
    print(features,features[8],features[-4],show_name)
    count += 1
    if count == 10:
        break

29 22 StutterTalk
11763    3
Name: NoStutteredWords, dtype: int64
[0 0 0 0 0 0 0 0 3 3 0 0] 3 3 StutterTalk
42 26 WomenWhoStutter
22660    3
Name: NoStutteredWords, dtype: int64
[0 0 0 0 0 0 0 2 3 0 0 0] 3 3 WomenWhoStutter
0 162 WomenWhoStutter
19176    1
Name: NoStutteredWords, dtype: int64
[0 0 0 0 2 0 1 1 1 0 0 3] 1 1 WomenWhoStutter
25 45 StutterTalk
11322    2
Name: NoStutteredWords, dtype: int64
[0 0 1 0 0 0 1 0 2 0 0 0] 2 2 StutterTalk
68 30 StutterTalk
14130    2
Name: NoStutteredWords, dtype: int64
[0 0 0 1 0 0 0 0 2 1 0 0] 2 2 StutterTalk
64 57 StutterTalk
13831    3
Name: NoStutteredWords, dtype: int64
[0 0 0 0 0 0 0 1 3 0 0 0] 3 3 StutterTalk
3 66 IStutterSoWhat
5025    0
Name: NoStutteredWords, dtype: int64
[0 0 0 1 3 0 0 1 0 0 0 0] 0 0 IStutterSoWhat
81 10 WomenWhoStutter
25569    1
Name: NoStutteredWords, dtype: int64
[0 1 0 1 0 2 0 0 1 0 0 0] 1 1 WomenWhoStutter
18 160 HeStutters
2540    2
Name: NoStutteredWords, dtype: int64
[0 1 0 1 0 0 0 2 2 0 0 0] 2 2 HeStutters
63

In [20]:
import os

root_folder = './sep28_all_together/'  # Replace with the actual path to your root folder

total_files = 0

# Walk through the root folder and its subfolders
for dirpath, dirnames, filenames in os.walk(root_folder):
    total_files += len(filenames)

print("Total number of files:", total_files)


Total number of files: 21856


In [15]:
import os
import tensorflow as tf

# Specify the source directory where your files are located
folder_path = './sep28_all_together/'

for filename in os.listdir(folder_path):

    path_parts = tf.strings.split(filename, '/')
    path_end = path_parts[-1]
    file_parts = tf.strings.split(path_end, '_')
    show_name = file_parts[0].numpy().decode('utf-8')
    pod_id = int(file_parts[-2])
    clip_id = int(tf.strings.split(file_parts[-1], '.')[-2])
    features = df_sep[(df_sep['Show'] == show_name) & (df_sep['EpId'] == pod_id) & (df_sep['ClipId'] == clip_id)].to_numpy()[0][5:]

    
    if features[8] > 2:
        old_file_path = os.path.join(folder_path, filename)
        new_filename = "nostutter_" + filename  # Replace "new_name_" with your desired naming convention
        new_file_path = os.path.join(folder_path, new_filename)
        os.rename(old_file_path, new_file_path)

    else:
        old_file_path = os.path.join(folder_path, filename)
        new_filename = "stutter_" + filename  # Replace "new_name_" with your desired naming convention
        new_file_path = os.path.join(folder_path, new_filename)
        os.rename(old_file_path, new_file_path)

In [16]:
import os

# Specify the directory where your .wav files are located
directory = './sep28_all_together/'

# Initialize a counter variable
count = 0
count1 = 0
# Iterate over the files in the directory
for filename in os.listdir(directory):
    if filename.startswith('nostutter'):
        count += 1
    if filename.startswith('stutter'):
        count1 += 1

print(f"Number of nostutter and stutter files: {count} , {count1} and tot {count+count1}")

Number of nostutter and stutter files: 6280 , 15576 and tot 21856


In [13]:
!ls ./sep28_all_together/stutter*

./sep28_all_together/stutter_WomenWhoStutter_87_118.wav
./sep28_all_together/stutter_WomenWhoStutter_87_119.wav
./sep28_all_together/stutter_WomenWhoStutter_87_11.wav
./sep28_all_together/stutter_WomenWhoStutter_87_120.wav
./sep28_all_together/stutter_WomenWhoStutter_87_121.wav
./sep28_all_together/stutter_WomenWhoStutter_87_122.wav
./sep28_all_together/stutter_WomenWhoStutter_87_123.wav
./sep28_all_together/stutter_WomenWhoStutter_87_124.wav
./sep28_all_together/stutter_WomenWhoStutter_87_126.wav
./sep28_all_together/stutter_WomenWhoStutter_87_127.wav
./sep28_all_together/stutter_WomenWhoStutter_87_128.wav
./sep28_all_together/stutter_WomenWhoStutter_87_129.wav
./sep28_all_together/stutter_WomenWhoStutter_87_12.wav
./sep28_all_together/stutter_WomenWhoStutter_87_130.wav
./sep28_all_together/stutter_WomenWhoStutter_87_131.wav
./sep28_all_together/stutter_WomenWhoStutter_87_132.wav
./sep28_all_together/stutter_WomenWhoStutter_87_133.wav
./sep28_all_together/stutter_WomenWhoStutter_87_13

In [2]:
import random
import shutil
import os

# Specify the source directory where your files are located
source_directory = './sep28_all_together'

# Specify the destination directories for the split files
destination_directory_1 = './train_sep28'
destination_directory_2 = './val_sep28'
destination_directory_3 = './test_sep28'

# Create the destination directories if they don't exist
for directory in [destination_directory_1, destination_directory_2, destination_directory_3]:
    if not os.path.exists(directory):
        os.makedirs(directory)

# Shuffle the files in the source directory
files = os.listdir(source_directory)
random.shuffle(files)

# Define the sizes for each split
split_sizes = [0.8, 0.1, 0.1]  # 60% for directory_1, 30% for directory_2, 10% for directory_3

# Calculate the number of files for each split
total_files = len(files)
split_counts = [int(size * total_files) for size in split_sizes]

# Split the files into different lists based on the split counts
file_splits = [files[:split_counts[0]], files[split_counts[0]:split_counts[0] + split_counts[1]], files[-split_counts[2]:]]

# Move the files to the destination directories
for i, file_split in enumerate(file_splits):
    if i == 0:
        destination_directory = destination_directory_1
    elif i == 1:
        destination_directory = destination_directory_2
    else:
        destination_directory = destination_directory_3

    for filename in file_split:
        source_path = os.path.join(source_directory, filename)
        destination_path = os.path.join(destination_directory, filename)
        shutil.copy2(source_path, destination_path)


In [12]:
# Specify the directory where your .wav files are located
directory1 = './train_sep28'
directory2 = './val_sep28'
directory3 = './test_sep28'

# Initialize a counter variable
count_train = 0
count_val = 0
count_test = 0

count_train_tot = 0
count_val_tot = 0
count_test_tot = 0

# Iterate over the files in the directory
for filename in os.listdir(directory1):
    if filename.startswith('nostutter'):
        count_train += 1

    if filename.endswith('.wav'):
        count_train_tot += 1
    if not filename.endswith('.wav'):
        print('error')
    if not filename.startswith('nostutter') and not filename.startswith('stutter'):
        print(filename)

# Iterate over the files in the directory
for filename in os.listdir(directory2):
    if filename.startswith('nostutter'):
        count_val += 1

    if filename.endswith('.wav'):
        count_val_tot += 1
    
    if not filename.endswith('.wav'):
        print('error')
    if not filename.startswith('nostutter') and not filename.startswith('stutter'):
        print(filename)

# Iterate over the files in the directory
for filename in os.listdir(directory3):
    if filename.startswith('nostutter'):
        count_test += 1
        
    if filename.endswith('.wav'):
        count_test_tot += 1
    
    if not filename.endswith('.wav'):
        print('error')
    if not filename.startswith('nostutter') and not filename.startswith('stutter'):
        print(filename)

print('**********Sep28 dataset**********')
print(f"Number of prolongation files in the train folder: {count_train} and tot is {count_train_tot}")
print(f"Number of prolongation files in the val folder: {count_val} and tot is {count_val_tot}")
print(f"Number of prolongation files in the test folder: {count_test} and tot is {count_test_tot}")

**********Sep28 dataset**********
Number of prolongation files in the train folder: 5043 and tot is 17484
Number of prolongation files in the val folder: 633 and tot is 2185
Number of prolongation files in the test folder: 604 and tot is 2185


In [36]:
# !ls ./sep28_all_together/*MyStutteringLife*
import glob

folder_path = './sep28_all_together'  # Specify the path to the folder
file_pattern = 'nostutter_MyStutteringLife*'  # Specify the file pattern or extension

# Use glob to find files matching the specified pattern
files = glob.glob(folder_path + '/' + file_pattern)

num_files = len(files)  # Get the count of files

print(f"Number of '{file_pattern}' files in the folder: {num_files}")




Number of 'nostutter_MyStutteringLife*' files in the folder: 851


In [ ]:
import os
import random

folder_path = './folder'  # Specify the path to the folder
num_files_to_keep = 850  # Specify the number of files to keep

# Get a list of all files in the folder
files = os.listdir(folder_path)

# Shuffle the list of files randomly
random.shuffle(files)

# Determine the files to delete
files_to_delete = files[num_files_to_keep:]

# Delete the files
for file in files_to_delete:
    file_path = os.path.join(folder_path, file)
    os.remove(file_path)

print(f"{len(files_to_delete)} files deleted. {num_files_to_keep} files kept.")


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=5c74a7a1-d505-4b2b-a4fa-efea0c0f20da' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>